In [1]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import string
import re
from bs4 import BeautifulSoup
import requests
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [2]:
directory = 'data'
all_data = []
years = []
for file in os.listdir(directory)[3:]:
    filename = file.split('.')[0]
    year = filename.split('-')[-1]
    years.extend([year]*100)
    with open(os.path.join(directory, file), 'r') as f:
        all_data.append(f.name)

all_data

['data\\songs-1970.csv',
 'data\\songs-1971.csv',
 'data\\songs-1972.csv',
 'data\\songs-1973.csv',
 'data\\songs-1974.csv',
 'data\\songs-1975.csv',
 'data\\songs-1976.csv',
 'data\\songs-1977.csv',
 'data\\songs-1978.csv',
 'data\\songs-1979.csv',
 'data\\songs-1980.csv',
 'data\\songs-1981.csv',
 'data\\songs-1982.csv',
 'data\\songs-1983.csv',
 'data\\songs-1984.csv',
 'data\\songs-1985.csv',
 'data\\songs-1986.csv',
 'data\\songs-1987.csv',
 'data\\songs-1988.csv',
 'data\\songs-1989.csv',
 'data\\songs-1990.csv',
 'data\\songs-1991.csv',
 'data\\songs-1992.csv',
 'data\\songs-1993.csv',
 'data\\songs-1994.csv',
 'data\\songs-1995.csv',
 'data\\songs-1996.csv',
 'data\\songs-1998.csv',
 'data\\songs-1999.csv',
 'data\\songs-2000.csv',
 'data\\songs-2001.csv',
 'data\\songs-2002.csv',
 'data\\songs-2003.csv',
 'data\\songs-2005.csv',
 'data\\songs-2006.csv',
 'data\\songs-2007.csv',
 'data\\songs-2008.csv',
 'data\\songs-2009.csv',
 'data\\songs-2010.csv',
 'data\\songs-2011.csv',


In [3]:
df = pd.concat(map(pd.read_csv, all_data), ignore_index=True)
songs_df = df.drop(['Unnamed: 0'], axis=1)
# add the column for year of appearance in the chart
songs_df['year'] = years

print(songs_df.columns)
songs_df

Index(['id', 'title', 'primary_artist', 'all_artists', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'duration_s', 'year'],
      dtype='object')


,id,title,primary_artist,all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_s,year
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,['The Beatles'],0.443,0.403,0,-8.339,1,0.0322,0.6310,0.000000,0.1110,0.4100,143.462,4,243,1970
1,6QhXQOpyYvbpdbyjgAqKdY,Cecilia,Simon & Garfunkel,['Simon & Garfunkel'],0.755,0.876,0,-8.867,1,0.0362,0.3570,0.000005,0.2200,0.9540,102.762,4,175,1970
2,2BhrrVRC8CoiWejbzj0VjQ,Son of a Preacher Man,Aretha Franklin,['Aretha Franklin'],0.474,0.473,2,-11.454,1,0.0601,0.5450,0.000001,0.0356,0.5610,77.583,4,199,1970
3,3Bh6uInhcVBVvLraGZdkKD,Who'll Stop The Rain,Creedence Clearwater Revival,['Creedence Clearwater Revival'],0.598,0.797,7,-6.793,1,0.0332,0.0424,0.000407,0.0717,0.6220,123.566,4,147,1970
4,6UkMcAA19lTdjs22jtB7o2,Big Yellow Taxi,Joni Mitchell,['Joni Mitchell'],0.611,0.470,4,-9.135,1,0.0356,0.5790,0.000000,0.5810,0.9700,85.527,4,135,1970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,7AODNxFNPnKaz4hURLzw5l,Ladbroke Grove,AJ Tracey,['AJ Tracey'],0.902,0.839,11,-9.447,0,0.2080,0.0933,0.000000,0.1020,0.7270,133.989,4,191,2019
4796,44ADyYoY5liaRa3EOAl4uf,Slide Away,Miley Cyrus,['Miley Cyrus'],0.530,0.538,5,-6.672,1,0.0335,0.1700,0.000003,0.2480,0.2880,148.051,4,234,2019
4797,2KKijf6Ett4kmVNJze2roh,The Git Up,Blanco Brown,['Blanco Brown'],0.847,0.678,9,-8.635,1,0.1090,0.0669,0.000000,0.2740,0.8110,97.984,4,201,2019
4798,2bjUEg4jBtKBlPdNrTAppI,Easier,5 Seconds of Summer,['5 Seconds of Summer'],0.562,0.460,5,-4.173,1,0.2590,0.4760,0.000000,0.1070,0.6230,176.055,4,158,2019


In [ ]:
## Get genre
load_dotenv()
cid = os.getenv('client_id')
secret = os.getenv('client_secret')

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

chars = re.escape(string.punctuation)
songs_df = songs_df.assign(genres = '')

def get_genre_by_artist(df):
    for artist in df['primary_artist'].unique():
        artistname = re.sub(r'['+chars+']', '', artist) if artist in ["T'Pau", "Des'ree", "K'NAAN", "Rag'n'Bone Man"] else artist
        clean_artistname = artistname.replace('&', 'and') if '&' in artistname else artistname
        try:  
            results = sp.search(q='artist:' + clean_artistname, type='artist')
            genres = results['artists']['items'][0]['genres']

            artist_idx = df[df['primary_artist'] == str(artist)].index
            for idx in artist_idx:
                df.at[idx, 'genres'] = genres
        except:
            print('error at', artist)
        
get_genre_by_artist(songs_df)

In [ ]:
def get_lyrics_by_song(df):
    for index, row in df.iterrows():
        artist = df.iloc[index]['primary_artist']
        song = df.iloc[index]['title']

        # https://stackoverflow.com/questions/23996118/replace-special-characters-in-a-string-in-python
        artistname = artist.replace('&', 'and') if '&' in artist else artist
        clean_artistname = re.sub(r'['+chars+']', '', artistname)
        artistname_url =  clean_artistname.replace(' ','-') if ' ' in clean_artistname else clean_artistname
#         songname = re.sub(r'['+chars+']', '', (song.split(' - ')[0].split("[")[0].split("(")[0]))
        songname = re.sub(r'['+chars+']', '', (song.split(' - ')[0].split("[")[0].split(",")[0]))
        clean_songname = songname.replace(' ','-') if ' ' in songname else songname
        songname_url = clean_songname[:-1] if clean_songname[-1:] == '-' else clean_songname

        page = requests.get('https://genius.com/'+ artistname_url + '-' + songname_url + '-' + 'lyrics')
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics_container = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-6 YYrds")

        if lyrics_container: 
            lyrics = lyrics_container.text
        elif lyrics_container == None:
            lyrics = None
            print(df.at[index,'primary_artist'],':', df.at[index,'title'])

        df.at[index, 'lyrics'] = lyrics
        
get_lyrics_by_song(songs_df)

In [ ]:
with open('./data/merge_with_lyrics_and_genres.csv', 'w', encoding='utf-8') as f:
        songs_df.to_csv(f, header= True, index=False)